In [2]:
import MDAnalysis as mda
from tqdm import trange, tqdm
import numpy as np
import matplotlib.pyplot as plt

/home/f0042vb/anaconda3/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The initial configuration is generated from a crystal structure (CCDC2053567). CHARMM36 was used as force field. Tip3p was used as water model.

We used two starting configurations (with guests pointing at different orientations).

<img src="./figure 1.png" alt="image1" width="20%" height="auto">
<img src="./figure 2.png" alt="image1" width="23%" height="auto">

Now, we have to generate the bond restrictions from the initial configuration

In [8]:
from MDAnalysis.analysis.distances import distance_array
from MDAnalysis.lib.distances import calc_angles, calc_dihedrals

def calculate_interactions(u, bond_indices, angle_indices, dihedral_indices):
    # Load your molecular system

    # Select atoms based on indices (0-based indexing in MDAnalysis)
    atoms = [u.atoms[idx - 1] for idx in bond_indices + angle_indices + dihedral_indices]

    # Initialize lists to store calculated values
    distances = []
    angles = []
    dihedrals = []


    # Calculate distance for bonds
    dist = distance_array(atoms[0].position, atoms[1].position)[0][0]/10
    distances.append(dist)

    # Calculate angles in degrees
    angle1 = np.degrees(calc_angles(atoms[2].position.reshape(1, 3), atoms[0].position.reshape(1, 3), atoms[1].position.reshape(1, 3))[0])
    angle2 = np.degrees(calc_angles(atoms[0].position.reshape(1, 3), atoms[1].position.reshape(1, 3), atoms[3].position.reshape(1, 3))[0])
    angles.append((angle1, angle2))

    # Calculate dihedrals in degrees
    dihedral1 = np.degrees(calc_dihedrals(atoms[4].position.reshape(1, 3), atoms[2].position.reshape(1, 3), atoms[0].position.reshape(1, 3), atoms[1].position.reshape(1, 3))[0])
    dihedral2 = np.degrees(calc_dihedrals(atoms[2].position.reshape(1, 3), atoms[0].position.reshape(1, 3), atoms[1].position.reshape(1, 3), atoms[3].position.reshape(1, 3))[0])
    dihedral3 = np.degrees(calc_dihedrals(atoms[0].position.reshape(1, 3), atoms[1].position.reshape(1, 3), atoms[3].position.reshape(1, 3), atoms[5].position.reshape(1, 3))[0])
    dihedrals.append((dihedral1, dihedral2, dihedral3))

    # Print the results
    print("[ bonds ]")
    print("; ai     aj    type   bA      kA     bB      kB")
    print(f" {bond_indices[0]}   {bond_indices[1]}       6    {distances[0]:.3f}   0.0    {distances[0]:.3f}   4184.0")

    print("\n[ angles ]")
    print("; ai     aj    ak     type    thA      fcA        thB      fcB")
    print(f" {angle_indices[0]}  {bond_indices[0]}   {bond_indices[1]}     1       {angles[0][0]:.2f}     0.0        {angles[0][0]:.2f}     41.84")
    print(f" {bond_indices[0]}  {bond_indices[1]}     {angle_indices[1]}   1       {angles[0][1]:.2f}     0.0        {angles[0][1]:.2f}     41.84")

    print("\n[ dihedrals ]")
    print("; ai     aj    ak    al    type     thA      fcA       thB      fcB")
    print(f" {dihedral_indices[0]}  {angle_indices[0]}   {bond_indices[0]}   {bond_indices[1]}      2       {dihedrals[0][0]:.2f}    0.0    {dihedrals[0][0]:.2f}    41.84")
    print(f" {angle_indices[0]}  {bond_indices[0]}   {bond_indices[1]}     {angle_indices[1]}    2       {dihedrals[0][1]:.2f}    0.0    {dihedrals[0][1]:.2f}    41.84")
    print(f" {bond_indices[0]}  {bond_indices[1]}     {angle_indices[1]}   {dihedral_indices[1]}    2       {dihedrals[0][2]:.2f}    0.0    {dihedrals[0][2]:.2f}    41.84")

tpr = f"/home/f0042vb/CHEM101.6/final_proj/complex_integration_meoh/em.tpr"
gro = f"/home/f0042vb/CHEM101.6/final_proj/complex_integration_meoh/complex.gro"

u = mda.Universe(tpr,gro)

bond_indices = [149, 48]
angle_indices = [150, 114]
dihedral_indices = [153, 112]

calculate_interactions(u, bond_indices, angle_indices, dihedral_indices)

[ bonds ]
; ai     aj    type   bA      kA     bB      kB
 149   48       6    0.352   0.0    0.352   4184.0

[ angles ]
; ai     aj    ak     type    thA      fcA        thB      fcB
 150  149   48     1       93.15     0.0        93.15     41.84
 149  48     114   1       106.38     0.0        106.38     41.84

[ dihedrals ]
; ai     aj    ak    al    type     thA      fcA       thB      fcB
 153  150   149   48      2       159.87    0.0    159.87    41.84
 150  149   48     114    2       53.55    0.0    53.55    41.84
 149  48     114   112    2       47.57    0.0    47.57    41.84


After running the simulation for 30 $\lambda$ for complex, and 20 $\lambda$ for the ligand, we summarize all dhdl.xvg in two folders. Now let's use alchemlyb package (https://github.com/alchemistry/alchemlyb) and MBAR package in it.

In [3]:
import os
import alchemlyb
from alchemlyb.parsing.gmx import extract_u_nk
from alchemlyb.estimators import MBAR

folder = '/home/f0042vb/CHEM101.6/final_proj/complex_integration_meoh/dhdl'
xvg_list = []
for filename in sorted(os.listdir(folder)):
    if filename.endswith('.xvg'):
        # Define the string

        pattern = r"\d+"
        full_path = os.path.join(folder, filename)

        xvg_list.append(full_path)

u_nk_complex_in = alchemlyb.concat([extract_u_nk(xvg, T=300) for xvg in xvg_list])
mbar_comp_in = MBAR().fit(u_nk_complex_in)
mbar_comp_in.delta_f_

Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.

****** PyMBAR will use 64-bit JAX! *******
* JAX is currently set to 32-bit bitsize *
* which is its default.                  *
*                                        *
* PyMBAR requires 64-bit mode and WILL   *
* enable JAX's 64-bit mode when called.  *
*                                        *
* This MAY cause problems with other     *
* Uses of JAX in the same code.          *
******************************************


******* JAX 64-bit mode is now on! *******
*     JAX is now set to 64-bit mode!     *
*   This MAY cause problems with other   *
*      uses of JAX in the same code.     *
******************

,"(0.0, 0.0, 0.0)","(0.0, 0.0, 0.01)","(0.0, 0.0, 0.025)","(0.0, 0.0, 0.05)","(0.0, 0.0, 0.075)","(0.0, 0.0, 0.1)","(0.0, 0.0, 0.2)","(0.0, 0.0, 0.35)","(0.0, 0.0, 0.5)","(0.0, 0.0, 0.75)",...,"(1.0, 0.5, 1.0)","(1.0, 0.6, 1.0)","(1.0, 0.65, 1.0)","(1.0, 0.7, 1.0)","(1.0, 0.75, 1.0)","(1.0, 0.8, 1.0)","(1.0, 0.85, 1.0)","(1.0, 0.9, 1.0)","(1.0, 0.95, 1.0)","(1.0, 1.0, 1.0)"
"(0.0, 0.0, 0.0)",0.000000,0.677489,1.388795,2.194662,2.790642,3.273358,4.600993,5.817306,6.704450,7.862681,...,26.315194,28.273601,29.280950,30.283936,31.240542,31.864239,31.409020,30.521313,30.170891,30.547852
"(0.0, 0.0, 0.01)",-0.677489,0.000000,0.711306,1.517174,2.113154,2.595869,3.923505,5.139817,6.026962,7.185193,...,25.637706,27.596113,28.603461,29.606447,30.563053,31.186750,30.731532,29.843824,29.493403,29.870363
"(0.0, 0.0, 0.025)",-1.388795,-0.711306,0.000000,0.805868,1.401847,1.884563,3.212198,4.428511,5.315655,6.473886,...,24.926400,26.884806,27.892155,28.895141,29.851747,30.475444,30.020225,29.132518,28.782096,29.159057
"(0.0, 0.0, 0.05)",-2.194662,-1.517174,-0.805868,0.000000,0.595980,1.078695,2.406331,3.622643,4.509788,5.668019,...,24.120532,26.078939,27.086287,28.089273,29.045880,29.669576,29.214358,28.326650,27.976229,28.353189
"(0.0, 0.0, 0.075)",-2.790642,-2.113154,-1.401847,-0.595980,0.000000,0.482715,1.810351,3.026663,3.913808,5.072039,...,23.524552,25.482959,26.490308,27.493293,28.449900,29.073596,28.618378,27.730671,27.380249,27.757209
"(0.0, 0.0, 0.1)",-3.273358,-2.595869,-1.884563,-1.078695,-0.482715,0.000000,1.327635,2.543948,3.431092,4.589323,...,23.041837,25.000243,26.007592,27.010578,27.967184,28.590881,28.135662,27.247955,26.897534,27.274494
"(0.0, 0.0, 0.2)",-4.600993,-3.923505,-3.212198,-2.406331,-1.810351,-1.327635,0.000000,1.216313,2.103457,3.261688,...,21.714201,23.672608,24.679957,25.682943,26.639549,27.263246,26.808027,25.920320,25.569898,25.946859
"(0.0, 0.0, 0.35)",-5.817306,-5.139817,-4.428511,-3.622643,-3.026663,-2.543948,-1.216313,0.000000,0.887145,2.045376,...,20.497889,22.456295,23.463644,24.466630,25.423236,26.046933,25.591714,24.704007,24.353586,24.730546
"(0.0, 0.0, 0.5)",-6.704450,-6.026962,-5.315655,-4.509788,-3.913808,-3.431092,-2.103457,-0.887145,0.000000,1.158231,...,19.610744,21.569151,22.576500,23.579485,24.536092,25.159788,24.704570,23.816863,23.466441,23.843401
"(0.0, 0.0, 0.75)",-7.862681,-7.185193,-6.473886,-5.668019,-5.072039,-4.589323,-3.261688,-2.045376,-1.158231,0.000000,...,18.452513,20.410920,21.418269,22.421254,23.377861,24.001558,23.546339,22.658632,22.308210,22.685170


Calculate the rest of the free energy differences:

In [13]:
folder = '/home/f0042vb/CHEM101.6/final_proj/complex_integration_meoh_out/dhdl'
xvg_list = []
for filename in sorted(os.listdir(folder)):
    if filename.endswith('.xvg'):
        # Define the string

        pattern = r"\d+"
        full_path = os.path.join(folder, filename)

        xvg_list.append(full_path)

u_nk_complex_out = alchemlyb.concat([extract_u_nk(xvg, T=300) for xvg in xvg_list])
mbar_comp_out = MBAR().fit(u_nk_complex_out)

folder = '/home/f0042vb/CHEM101.6/final_proj/ligand_integration_meoh/dhdl'
xvg_list = []
for filename in sorted(os.listdir(folder)):
    if filename.endswith('.xvg'):
        # Define the string

        pattern = r"\d+"
        full_path = os.path.join(folder, filename)

        xvg_list.append(full_path)

u_nk_ligand = alchemlyb.concat([extract_u_nk(xvg, T=300) for xvg in xvg_list])
mbar_ligand = MBAR().fit(u_nk_ligand)


Calculate the restriction free energy

In [22]:
import math
K = 8.314472*0.001  # Gas constant in kJ/mol/K
V = 1.66            # standard volume in nm^3

T      = 300.0      # Temperature in Kelvin
r0     = 0.352     # Distance in nm
thA    = 93.15       # Angle in degrees 
thB    = 106.38     # Angle in degrees

K_r    = 4184.0     # force constant for distance (kJ/mol/nm^2)
K_thA  = 41.84      # force constant for angle (kJ/mol/rad^2)
K_thB  = 41.84      # force constant for angle (kJ/mol/rad^2)
K_phiA = 41.84      # force constant for dihedral (kJ/mol/rad^2)
K_phiB = 41.84      # force constant for dihedral (kJ/mol/rad^2)
K_phiC = 41.84      # force constant for dihedral (kJ/mol/rad^2)          



thA = math.radians(thA)  # convert angle from degrees to radians --> math.sin() wants radians
thB = math.radians(thB)  # convert angle from degrees to radians --> math.sin() wants radians

arg =(
    (8.0 * math.pi**2.0 * V) / (r0**2.0 * math.sin(thA) * math.sin(thB)) 
    * 
    (
        ( (K_r * K_thA * K_thB * K_phiA * K_phiB * K_phiC)**0.5 ) / ( (2.0 * math.pi * K * T)**(3.0) )
    )
)

dGr_in = K * T * math.log(arg)

In [23]:

r0     = 0.637     # Distance in nm
thA    = 53.41       # Angle in degrees 
thB    = 102.98     # Angle in degrees

K_r    = 4184.0     # force constant for distance (kJ/mol/nm^2)
K_thA  = 41.84      # force constant for angle (kJ/mol/rad^2)
K_thB  = 41.84      # force constant for angle (kJ/mol/rad^2)
K_phiA = 41.84      # force constant for dihedral (kJ/mol/rad^2)
K_phiB = 41.84      # force constant for dihedral (kJ/mol/rad^2)
K_phiC = 41.84      # force constant for dihedral (kJ/mol/rad^2)          



thA = math.radians(thA)  # convert angle from degrees to radians --> math.sin() wants radians
thB = math.radians(thB)  # convert angle from degrees to radians --> math.sin() wants radians

arg =(
    (8.0 * math.pi**2.0 * V) / (r0**2.0 * math.sin(thA) * math.sin(thB)) 
    * 
    (
        ( (K_r * K_thA * K_thB * K_phiA * K_phiB * K_phiC)**0.5 ) / ( (2.0 * math.pi * K * T)**(3.0) )
    )
)

dGr_out = K * T * math.log(arg)


Calculate the final result:

In [33]:
dG_in = (mbar_ligand.delta_f_.iloc[0,-1]-mbar_comp_in.delta_f_.iloc[0,-1])*K*T+dGr_in
dG_out = (mbar_ligand.delta_f_.iloc[0,-1]-mbar_comp_out.delta_f_.iloc[0,-1])*K*T+dGr_out

# Calculate the error

d_dG_in = np.sqrt(mbar_ligand.d_delta_f_.iloc[0,-1]**2 + mbar_comp_in.d_delta_f_.iloc[0,-1]**2)*K*T
d_dG_out = np.sqrt(mbar_ligand.d_delta_f_.iloc[0,-1]**2 + mbar_comp_out.d_delta_f_.iloc[0,-1]**2)*K*T

print(dG_in, d_dG_in)
print(dG_out, d_dG_out)

-29.727928800616787 0.20453002139932341
-31.448105905210692 0.19827942166841755


Thus, the "in" configuration has a free energy difference: $$\Delta G = -29.72 \pm 0.20 \ kJ/mol$$
the "out" configuration has a free energy difference: $$\Delta G = -31.44 \pm 0.20 \ kJ/mol$$

If the possibility of two state is 50%, then the observed binding free energy $$\Delta G_{obs} = -30.73 \pm 0.15 \ kJ/mol$$